In [8]:
import torch
import optuna
import time
import copy
from torch import nn as nn
import torch.backends.cudnn as cudnn
from torch.optim import lr_scheduler
from pathlib import Path
import mlflow
from optuna.integration.mlflow import MLflowCallback
from thermostability.thermo_dataset import ThermostabilityDataset
from thermostability.hotinfer import HotInfer

cudnn.benchmark = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    torch.cuda.empty_cache() 
    
cpu = torch.device("cpu")

torch.cuda.list_gpu_processes()

'pynvml module not found, please install pynvml'

In [9]:
def prep_minimal_dataset(src_path: Path, dst_path: Path):
    minimized_data = []
    reduction = 10
    with open(src_path) as file:
        data = file.readlines()
        file_len = len(data)
        minimized_data = data[:round(len(data) / reduction)]
        print(f'Dataset reduced: {file_len} --> {len(minimized_data)}')
        
    with open(dst_path, 'w') as file:
        file.writelines(minimized_data)
            
prep_minimal_dataset(Path('data/train_sequences.fasta'), Path('data/minimal_train_sequences.fasta'))
prep_minimal_dataset(Path('data/eval_sequences.fasta'), Path('data/minimal_eval_sequences.fasta'))

Dataset reduced: 183697 --> 18370
Dataset reduced: 17585 --> 1758


In [10]:
train_ds = ThermostabilityDataset('data/minimal_train_sequences.fasta')
eval_ds = ThermostabilityDataset('data/minimal_eval_sequences.fasta')

dataloaders = {
    "train": torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(eval_ds, batch_size=32, shuffle=True, num_workers=4)
}

dataset_sizes = {"train": len(train_ds),"val": len(eval_ds)}

In [11]:
from tqdm.notebook import tqdm
import sys

def train_model(model, optimizer, criterion, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())

    best_epoch_loss = sys.float_info.max
    losses = []
    batchEnumeration = []
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
         

            # Iterate over data.
            for idx, (inputs, labels) in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    
                    loss = criterion(outputs,labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                batch_size = inputs.size(0)
                batch_loss = loss.item() * batch_size
                losses.append(batch_loss)
                batchEnumeration.append(batchEnumeration[-1]+1 if len(batchEnumeration)>0 else 0)

                running_loss += batch_loss
               
            
                if idx % 10 == 0:
                    batch_size = inputs.size(0)
                    tqdm.write("Epoch: [{}/{}], Batch: [{}/{}], train accuracy: {:.6f}, loss: {:.6f}".format(
                        epoch,
                        num_epochs,
                        idx + 1,
                        len(dataloaders[phase]),
                        batch_loss / float(batch_size)
                        ), end="\r")
                    
                    
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]


            print(f'{phase} Loss: {epoch_loss:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_loss < best_epoch_loss:
                best_epoch_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()


    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_epoch_loss:4f}')
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_epoch_loss

In [12]:
YOUR_TRACKING_URI = "http://127.0.0.1:5000"
mlflc = MLflowCallback(
    tracking_uri=YOUR_TRACKING_URI,
    metric_name="metric_score"
)

def optimize_thermostability(trial):    
    params = {
        'model_learning_rate': trial.suggest_float('model_learning_rate', 0.001, 0.501, step=0.05),
        'model_hidden_units': trial.suggest_int('model_hidden_units', 64, 640, step=64),
        'model_hidden_layers': trial.suggest_int('model_hidden_layers', 1, 4, step=1)
    }
    
    model = HotInfer(
        params['model_hidden_units'],
        params['model_hidden_layers'],
    )
    
    model.esmfold.requires_grad_(False)
    
    criterion = nn.MSELoss()

    optimizer_ft = torch.optim.SGD(model.parameters(), lr=params['model_learning_rate'], momentum=0.9)
    
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    model, score = train_model(model, optimizer_ft, criterion, exp_lr_scheduler, num_epochs=10)

    mlflow.log_params(params)
    eval_data = train_ds.thermo_dataframe.iloc[0,:]
    eval_data["label"] = train_ds.thermo_dataframe.iloc[1,:]
    candidate_model_uri = mlflow.pytorch.log_model(model).model_uri
    mlflow.evaluate(model=candidate_model_uri, data=eval_data, targets="label", model_type="regressor")
    mlflow.log_metric("score", score)
    return score


/home/leon/miniconda3/envs/hotprot/lib/python3.7/site-packages/ipykernel_launcher.py:4: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  after removing the cwd from sys.path.


In [13]:
# minimize or maximize
study = optuna.create_study(direction="minimize", study_name="thermostability-hyperparameter-optimization") # maximise the score during tuning
study.optimize(optimize_thermostability, n_trials=100) # run the objective function 100 times

print(study.best_trial) # print the best performing pipeline

[I 2022-12-24 17:41:18,151] A new study created in memory with name: thermostability-hyperparameter-optimization
[W 2022-12-24 17:41:18,239] Trial 0 failed because of the following error: ModuleNotFoundError("No module named 'openfold'")
Traceback (most recent call last):
  File "/home/leon/miniconda3/envs/hotprot/lib/python3.7/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_6242/774448019.py", line 16, in optimize_thermostability
    params['model_hidden_layers'],
  File "/home/leon/hot-prot/thermostability/hotinfer.py", line 16, in __init__
    self.esmfold = esm.pretrained.esmfold_v1()
  File "/home/leon/miniconda3/envs/hotprot/lib/python3.7/site-packages/esm/pretrained.py", line 419, in esmfold_v1
    import esm.esmfold.v1.pretrained
  File "/home/leon/miniconda3/envs/hotprot/lib/python3.7/site-packages/esm/esmfold/v1/pretrained.py", line 5, in <module>
    from esm.esmfold.v1.esmfold import ESMFold
  File "

ModuleNotFoundError: No module named 'openfold'